In [2]:
import metadata_utils as mu
from lxml import etree
from bs4 import BeautifulSoup
import os.path
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

THRESHOLD = 90


def get_page_ocrs(filepath):
    soup_file = open(filepath)
    print(f"now souping {filepath}")
    soup = BeautifulSoup(soup_file, features="xml")
    print("it has been souped")
    items = []
    for entry in soup.find_all("DLPSTEXTCLASS"):
        item_data = dict()
        item_data["id"] = entry.find("IDNO").text
        if item_data["id"] == "AAN6100.0001.00":
            print(entry)
        for page in entry.find_all("P")[:10]:
            page_ocr = page.text
            for page_info in page.find_all("PB"):
                    item_data[f"tpg - {page_info.attrs['SEQ']}"] = page_ocr
        items.append(item_data)
    return items

all_pages = get_page_ocrs("moaxml.nosync/moa1.xml") + get_page_ocrs("moaxml.nosync/moa2.xml") + get_page_ocrs("moaxml.nosync/moa3.xml")+ get_page_ocrs("moaxml.nosync/moa4.xml") + get_page_ocrs("moaxml.nosync/moa5.xml") + get_page_ocrs("moaxml.nosync/moa6.xml")


now souping moaxml.nosync/moa1.xml
it has been souped
now souping moaxml.nosync/moa2.xml
it has been souped
now souping moaxml.nosync/moa3.xml
it has been souped
now souping moaxml.nosync/moa4.xml
it has been souped
now souping moaxml.nosync/moa5.xml
it has been souped
now souping moaxml.nosync/moa6.xml
it has been souped


In [3]:
THRESHOLD = 90
import metadata_utils as mu
import pytesseract as pyt
import requests

def get_images(entry):
    print("starting get_images")
    counter = 0
    images = []
    counter
    for url in [f"https://quod.lib.umich.edu/m/moa/{entry[1]}/{x}rgn=full+text;view=image" for x in range(0,10)]:

        response = requests.get(url)
        if response and response.status_code == 200:
            page_soup = BeautifulSoup(response.text, features="html")
            for img in page_soup.find_all("img"):
                if img.attrs["alt"] == "Page image":
                    new_url = img.attrs["src"]
            if new_url:
                file_name = root + "_" + url.split("/")[-1][0]
                response = requests.get(f"https://quod.lib.umich.edu{new_url}")
                file_name = f"test_image_folder.nosync/{file_name}.jpg"
                images.append(file_name)
                with open(file_name, "wb") as f:
                    f.write(response.content)

    new_selection = process.extractOne(query=entry[3], choices=[pyt.image_to_string(img) for img in images], scorer=fuzz.partial_ratio)
    if new_selection[1] > THRESHOLD:
        print("Result: we have a winner")
        return True
    elif new_selection[1] < THRESHOLD:
        print("Result: we don't have a winner")
        return False


ids = mu.read_file("no_title_page_ids.csv")
print(len(ids))

passed1 = []
round_2 = []

pass_count = 0
r2_pass_count = 0
file_not_found_count = 0

#--round 1 testing (fuzzy check against existing ocr)--
for record in ids:
    if record[1] in [x["id"] for x in all_pages]:
        match = [x for x in all_pages if x["id"]==record[1]][0]
        selection, score = process.extractOne(query=record[3],choices=[key + "||" + val[:3 * len(record[3])] for key, val in match.items() if "tpg" in key], scorer=fuzz.partial_ratio)
        if score > THRESHOLD:
            pass_count += 1
            print("\nscore:", score,"title:", record[3],"\nid:", record[1],"\nselection:", selection,"\n ----------\n\n")
            passed1.append(record) #<---- to be added to confirmed list
        elif score < THRESHOLD:
            print("\nscore:", score,"title:", record[3],"\nid:", record[1],"\nselection:", selection, "\n ----------\n\n")
            #--round 2 testing (re-ocr image)--
        
            round_2.append(record) #<----- to be sent to round two
            root = record[1]
            page = selection.split("||")[0].strip("tpg - ")
            path = f"/n1/obj/{root[0]}/{root[1]}/{root[2]}/{root}/{page.strip('tpg - ')}.tif"

            new_result = get_images(entry=record)
            if new_result is True:
                r2_pass_count += 1
                print("new pass total", r2_pass_count)





            # #round_2_pathes.append(path)
            # # with open("need_filepathes.txt", 'a') as file:
            # #     file.write(path + "\n")
            # #-accessing filepaths from directory
            # img_files = os.listdir("moa_reocr_images.nosync")
            # img_path = root.lower() + "-" + page.strip("tpg - ")+".tif"
            # if img_path in img_files:
            #     new_score = process.extractOne(query=record[3], choices = [pyt.image_to_string("moa_reocr_images.nosync/" + img_path)], scorer=fuzz.partial_ratio)
            #     if new_score > THRESHOLD:
            #         print("A New Pass!")
            #         r2_pass_count += 1
            # else:
            #     file_not_found_count += 1



print("pass counter:", pass_count)
print("round2 num", len(round_2))
print("round2 pass", r2_pass_count)
print("r2 files not fount", file_not_found_count)
mu.write_csv("no_title_passed1.csv", passed1)
print("done")


384

score: 44 title: General register. 
id: AAF1187.1875.001 
selection: tpg - 00000003||
COURIER STEAM PRINTING HOUSE, ANN ARBOR, MICn.



 
 ----------


starting get_images
Result: we don't have a winner

score: 40 title: General register. 
id: AAF1187.1877.001 
selection: tpg - 00000004||
BOARD OF REGENTS.
JAMES B. ANGELL, LL. D.,
PRESIDE 
 ----------


starting get_images
Result: we don't have a winner

score: 38 title: General register. 
id: AAF1187.1881.001 
selection: tpg - 00000002||
CALENDAR
OF THE
UNIVERSITY OF MICHIGAN
FOR


1881- 
 ----------


starting get_images
Result: we don't have a winner

score: 50 title: Directory of social and health agencies of New York City. 
id: AAF3120.1911.001 
selection: tpg - 00000002||
II




l


Notice
This Directory is a guide to, and
not in any sense an indorsement of
the organizations entered in it.
While it is our purpose to exclude
those which ren 
 ----------


starting get_images
Result: we don't have a winner

score: 59 title: Di


Scores without limit on page characters :
- 100     : 61
- 99 - 90 : 133
- 89 - 80 : 42
- 79 - 70 : 25
- 69 - 60 : 35
- 59 - 50 : 46
- 49 - 40 : 24
- 39 - 30 : 12
- 29 - 20 : 4
- 19 - 10 : 1
-  9 - 0  : 0


Scores with limit on page characters :
- 100     : 48
- 99 - 90 : 132
- 89 - 80 : 45
- 79 - 70 : 26
- 69 - 60 : 26
- 59 - 50 : 43
- 49 - 40 : 36
- 39 - 30 : 21
- 29 - 20 : 5
- 19 - 10 : 1
-  9 - 0  : 0


TO GREG: Your next task, add these to the confirmed directory and infcoprate them with build_comfirmed.py
-no_title_passed1.csv
-fc_passed1.csv

look here: https://miro.com/app/board/uXjVP3IbLH8=/